### Problem Description 

* population = set of n individuals 
* an individual = one valid fantasy league
* Each league consists of 5 teams
* Each team is itself a list of 7 player dictionaries

<br> List of Individuals → Each Individual = List of 5 Teams → Each Team = List of 7 Players → Each Player = Dictionary

 Constraints:
 * every player must be assigned to exactly one team
 * each of the 5 teams must have 1 GK, 2 DEF, 2 MID, 2 FWD
 * the total salary of any team cannot exceed €750 million
 * all teams must be formed (no incompete teams nor not assigned players) 

In [1]:
# from initial_pop import *
from code_classes import *

In [5]:
players = load_players_from_csv("data/players(in).csv")
players

[Alex Carter (GK) Skill: 85 Salary: €90.0M,
 Jordan Smith (GK) Skill: 88 Salary: €100.0M,
 Ryan Mitchell (GK) Skill: 83 Salary: €85.0M,
 Chris Thompson (GK) Skill: 80 Salary: €80.0M,
 Blake Henderson (GK) Skill: 87 Salary: €95.0M,
 Daniel Foster (DEF) Skill: 90 Salary: €110.0M,
 Lucas Bennett (DEF) Skill: 85 Salary: €90.0M,
 Owen Parker (DEF) Skill: 88 Salary: €100.0M,
 Ethan Howard (DEF) Skill: 80 Salary: €70.0M,
 Mason Reed (DEF) Skill: 82 Salary: €75.0M,
 Logan Brooks (DEF) Skill: 86 Salary: €95.0M,
 Caleb Fisher (DEF) Skill: 84 Salary: €85.0M,
 Nathan Wright (MID) Skill: 92 Salary: €120.0M,
 Connor Hayes (MID) Skill: 89 Salary: €105.0M,
 Dylan Morgan (MID) Skill: 91 Salary: €115.0M,
 Hunter Cooper (MID) Skill: 83 Salary: €85.0M,
 Austin Torres (MID) Skill: 82 Salary: €80.0M,
 Gavin Richardson (MID) Skill: 87 Salary: €95.0M,
 Spencer Ward (MID) Skill: 84 Salary: €85.0M,
 Sebastian Perry (FWD) Skill: 95 Salary: €150.0M,
 Xavier Bryant (FWD) Skill: 90 Salary: €120.0M,
 Elijah Sanders 

We have to decide later what population size we want to set. By default i set 50.

In [7]:
initial_population = create_initial_population(players,population_size=50, salary_cap=750)
initial_population

[Team 1:
 Blake Henderson (GK) Skill: 87 Salary: €95.0M
 Jaxon Griffin (DEF) Skill: 79 Salary: €65.0M
 Ethan Howard (DEF) Skill: 80 Salary: €70.0M
 Nathan Wright (MID) Skill: 92 Salary: €120.0M
 Hunter Cooper (MID) Skill: 83 Salary: €85.0M
 Colton Gray (FWD) Skill: 91 Salary: €125.0M
 Elijah Sanders (FWD) Skill: 93 Salary: €140.0M
 Total Team Salary: €700.0M
 
 Team 2:
 Alex Carter (GK) Skill: 85 Salary: €90.0M
 Mason Reed (DEF) Skill: 82 Salary: €75.0M
 Caleb Fisher (DEF) Skill: 84 Salary: €85.0M
 Ashton Phillips (MID) Skill: 90 Salary: €110.0M
 Gavin Richardson (MID) Skill: 87 Salary: €95.0M
 Chase Murphy (FWD) Skill: 86 Salary: €95.0M
 Zachary Nelson (FWD) Skill: 86 Salary: €92.0M
 Total Team Salary: €642.0M
 
 Team 3:
 Jordan Smith (GK) Skill: 88 Salary: €100.0M
 Maxwell Flores (DEF) Skill: 81 Salary: €72.0M
 Brayden Hughes (DEF) Skill: 87 Salary: €100.0M
 Connor Hayes (MID) Skill: 89 Salary: €105.0M
 Bentley Rivera (MID) Skill: 88 Salary: €100.0M
 Adrian Collins (FWD) Skill: 85 Sa

In [8]:
for idx, individual in enumerate(initial_population, start=1):
    fitness = calculate_fitness(individual)
    print(f"Individual {idx} Fitness {fitness:.2f}")


Individual 1 Fitness 0.73
Individual 2 Fitness 1.10
Individual 3 Fitness 1.39
Individual 4 Fitness 1.74
Individual 5 Fitness 1.09
Individual 6 Fitness 1.56
Individual 7 Fitness 0.82
Individual 8 Fitness 1.07
Individual 9 Fitness 0.96
Individual 10 Fitness 1.13
Individual 11 Fitness 1.07
Individual 12 Fitness 0.95
Individual 13 Fitness 1.38
Individual 14 Fitness 1.14
Individual 15 Fitness 0.72
Individual 16 Fitness 0.81
Individual 17 Fitness 0.78
Individual 18 Fitness 0.70
Individual 19 Fitness 1.08
Individual 20 Fitness 1.12
Individual 21 Fitness 1.33
Individual 22 Fitness 0.55
Individual 23 Fitness 1.14
Individual 24 Fitness 0.92
Individual 25 Fitness 1.22
Individual 26 Fitness 0.85
Individual 27 Fitness 1.07
Individual 28 Fitness 1.31
Individual 29 Fitness 0.39
Individual 30 Fitness 0.85
Individual 31 Fitness 1.34
Individual 32 Fitness 1.24
Individual 33 Fitness 0.59
Individual 34 Fitness 1.11
Individual 35 Fitness 0.76
Individual 36 Fitness 1.24
Individual 37 Fitness 0.97
Individual